In [1]:
import pandas as pd

In [2]:
# List of columns to import (avoids attempting to import empty columns)
columns_to_import = ["Case Type","Region","Case Number","Case Name","Status","Date Filed","Date Closed","Reason Closed"
                     ,"City","States & Territories","Employees on charge/petition","Allegations","Participants","Union",
                     "Unit Sought","Voters"]

# Column data types, to avoid Pandas trying to parse data type on every column
column_dtypes = {c:"string" for c in columns_to_import}

In [3]:
# Import various partial data files, parsing dates in the date columns. All other columns are strings.
df1978_1999 = pd.read_csv("1978-1999.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")

# Note: broken entries in rows 10108 - 10111
df2000_2004 = pd.read_csv("2000-2004.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2005_2009 = pd.read_csv("2005-2009.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2010 = pd.read_csv("2010.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2011 = pd.read_csv("2011.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2012 = pd.read_csv("2012.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2013 = pd.read_csv("2013.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2014 = pd.read_csv("2014.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")

# Note: broken entry at row 1196 and 1197
df2015_2019 = pd.read_csv("2015-2019.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2020_2024 = pd.read_csv("2020-2024.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")

In [17]:
# As imported, data files are descending in date. Concatenate into one large data file.
# According to NLRB website, we should have 412094 cases, but we get 413889 rows. The overcount
# is probably due to rows that broke while downloading. TODO: check which years need to be redone.
unfair_labor_practices = pd.concat([df2020_2024, df2015_2019, df2014, df2013, df2012, df2011, df2010, df2005_2009, df2000_2004, df1978_1999], ignore_index = True)

# When imported, missing values became "<NA>". Replace these with NaT to match usual Pandas convention by 
# replacing "<NA>" by pd.NaT, then using to_datetime and coercing NaTs.
unfair_labor_practices["Date Closed"] = pd.to_datetime(unfair_labor_practices["Date Closed"].replace("<NA>", pd.NaT), errors = "coerce")
unfair_labor_practices["Date Filed"] = pd.to_datetime(unfair_labor_practices["Date Filed"].replace("<NA>", pd.NaT), errors = "coerce")

In [23]:
# Remove entries where case type is not "C" (removes accidentally included representation petitions,
# which are the vast majority of removed entries, plus some broken entries)
unfair_labor_practices = unfair_labor_practices[unfair_labor_practices["Case Type"] == "C"]

In [29]:
# Remove broken entries by checking case number formatting
unfair_labor_practices = unfair_labor_practices[unfair_labor_practices["Case Number"].str.match(r'..-..-*')]

In [31]:
# Print case counts by year.
print({year: unfair_labor_practices[unfair_labor_practices["Date Filed"].dt.year == year].shape[0] for year in range(2000,2025)})

{2000: 3919, 2001: 5164, 2002: 7595, 2003: 9135, 2004: 7252, 2005: 8911, 2006: 13048, 2007: 22987, 2008: 22190, 2009: 24849, 2010: 23671, 2011: 21469, 2012: 21216, 2013: 21334, 2014: 24152, 2015: 20044, 2016: 21032, 2017: 19146, 2018: 18891, 2019: 18343, 2020: 15139, 2021: 15534, 2022: 18609, 2023: 20305, 2024: 6099}


In [32]:
# Export the ones that match NLRBs website counts (pre-2000 in separate file)
unfair_labor_practices[unfair_labor_practices["Date Filed"].dt.year < 2000].drop_duplicates().to_csv("cleaned/pre2000.csv")
for year in [2000,2001,2002,2005, 2006,2007,2008, 2010, 2011, 2012, 2015, 2016, 2017, 2018, 2021]:
    filt = (unfair_labor_practices["Date Filed"].dt.year == year)
    unfair_labor_practices[filt].drop_duplicates().to_csv("cleaned/"+str(year)+".csv")

In [33]:
# Process manually downloaded CSVs for years that didn't match NLRB website.
# These CSVs also don't quite match the NLRB website count but at least are devoid
# of broken entries.
df2003 = pd.read_csv("2003.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2004 = pd.read_csv("2004.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2009 = pd.read_csv("2009.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2013 = pd.read_csv("2013.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2014 = pd.read_csv("2014.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2019 = pd.read_csv("2019.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2020 = pd.read_csv("2020.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2022 = pd.read_csv("2022.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2023 = pd.read_csv("2023.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")
df2024 = pd.read_csv("2024.csv", usecols = columns_to_import, dtype = column_dtypes, parse_dates = ["Date Filed", "Date Closed"], date_format = "%m%d%y")

In [38]:
unfair_labor_practices_fix = pd.concat([df2003,df2004,df2009,df2013,df2014,df2019,df2020,df2022,df2023,df2024], ignore_index = True)

# When imported, missing values became "<NA>". Replace these with NaT to match usual Pandas convention by 
# replacing "<NA>" by pd.NaT, then using to_datetime and coercing NaTs.
unfair_labor_practices_fix["Date Closed"] = pd.to_datetime(unfair_labor_practices_fix["Date Closed"].replace("<NA>", pd.NaT), errors = "coerce")
unfair_labor_practices_fix["Date Filed"] = pd.to_datetime(unfair_labor_practices_fix["Date Filed"].replace("<NA>", pd.NaT), errors = "coerce")

In [42]:
# Remove accidentally included representation petitions
unfair_labor_practices_fix = unfair_labor_practices_fix[unfair_labor_practices_fix["Case Type"] == "C"]

In [46]:
# Remove broken entries by checking case number formatting
unfair_labor_practices_fix = unfair_labor_practices_fix[unfair_labor_practices_fix["Case Number"].str.match(r'..-..-*')]

In [48]:
# Export remaining years
for year in [2003, 2004, 2009, 2013, 2014, 2019, 2020, 2022, 2023, 2024]:
    filt = (unfair_labor_practices_fix["Date Filed"].dt.year == year)
    unfair_labor_practices_fix[filt].drop_duplicates().to_csv("cleaned/"+str(year)+".csv")